In [25]:
%matplotlib inline
from PIL import Image
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LogisticRegression
import os
import itertools
from tqdm import *
from tqdm import tqdm_notebook as tqdm
import random
import numpy as np
import math

from IPython.display import clear_output

## Create pairs of filenames and truth values from training data

In [26]:
fileNames = []
for file in os.listdir("data/train_1"):
    if file.endswith(".jpg"):
        fileNames.append(file)
# fileNamesDF = pd.DataFrame(fileNames)

Exception in thread Thread-7:
Traceback (most recent call last):
  File "/Users/adriencogny/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Users/adriencogny/anaconda3/lib/python3.6/site-packages/tqdm/_tqdm.py", line 148, in run
    for instance in self.tqdm_cls._instances:
  File "/Users/adriencogny/anaconda3/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



In [27]:
def isPaintingFromPopularArtist(paintingRow,popularArtists):
    print(paintingRow)
    print(paintingRow[artist])
    return 0

In [28]:
#only take certain artist so that the probabilities aren't basically all false when doing the tupples
minNumPaintingsPerArtist = 400
trainInfo = pd.read_csv('data/train_info.csv')
mostPopularArtists = trainInfo['artist'].value_counts()[trainInfo['artist'].value_counts() > minNumPaintingsPerArtist]
fileNamesDF = trainInfo[trainInfo['artist'].isin(mostPopularArtists.index)]['filename']
# print(trainInfo[trainInfo['filename'].isin(fileNames)].shape)

In [29]:
fileNamesDF.shape

(1216,)

In [30]:
fractionOfData = 0.03
trainFiles = fileNamesDF.sample(frac=fractionOfData)
trainFileNames = trainFiles.values.flatten()

In [31]:
## from itertools documentation to create tupples
def product(*args, repeat=1):
    # product('ABCD', 'xy') --> Ax Ay Bx By Cx Cy Dx Dy
    # product(range(2), repeat=3) --> 000 001 010 011 100 101 110 111
    pools = [tuple(pool) for pool in args] * repeat
    result = [[]]
    for pool in pools:
        result = [x+[y] for x in result for y in pool]
    for prod in result:
        yield tuple(prod)
def permutations(iterable, r=None):
    pool = tuple(iterable)
    n = len(pool)
    r = n if r is None else r
    for indices in product(range(n), repeat=r):
        if len(set(indices)) == r:
            yield tuple(pool[i] for i in indices)
def combinations(iterable, r):
    pool = tuple(iterable)
    n = len(pool)
    for indices in permutations(range(n), r):
        if sorted(indices) == list(indices):
            yield tuple(pool[i] for i in indices)

def isSameArtist(image1,image2,trainInfo):
    artist1 = trainInfo[trainInfo['filename'] == image1]['artist']
    artist2 = trainInfo[trainInfo['filename'] == image2]['artist']
    return artist1.iloc[0] == artist2.iloc[0]

In [32]:
def generateTrainingSet(trainFileNames,fractionOfData,minNumPaintingsPerArtist,keepTQDM = True):
    print("Generating Training Set for " + str(len(trainFileNames)) + " files.")
    trainInputs = pd.DataFrame(list(combinations(trainFileNames,2)),columns=['image1','image2'])
    trainInputs.to_csv('trainingTupplesNoTruth'+ str(fractionOfData) + '-' + str(minNumPaintingsPerArtist) +'.csv')
    trainInfo = pd.read_csv('data/train_info.csv')
    trainInputs['sameArtist'] = pd.Series([isSameArtist(x[0],x[1],trainInfo) for x in tqdm(trainInputs.values,leave=keepTQDM)])
    trainInputs.to_csv('trainingTupples' + str(fractionOfData) + '-' + str(minNumPaintingsPerArtist) + '.csv')
    return trainInputs

In [33]:
trainingInputs = generateTrainingSet(trainFileNames,fractionOfData,minNumPaintingsPerArtist)

Generating Training Set for 36 files.


HBox(children=(IntProgress(value=0, max=630), HTML(value='')))

In [34]:
PercentageTrue = trainingInputs[trainingInputs['sameArtist'] == True].shape[0] / trainingInputs.shape[0]
PercentageFalse = 1 - PercentageTrue

In [35]:
print(PercentageTrue)
print(PercentageFalse)

0.3396825396825397
0.6603174603174603


## Probabilistic Model

In [36]:
def score(testingInputs):
    equality = testingInputs['sameArtist'] == testingInputs['predictions']
    numEqual = equality[equality == True].shape[0]
    numTotal = equality.shape[0]
    return numEqual / numTotal

In [37]:
def runProbModel(tuppleFrame,probTrue):
    returnedValues = []
    returnedPredictions = []
    for i in range(tuppleFrame.shape[0]):
        value = random.random()
        returnedValues.append(value)
        returnedPredictions.append(value < probTrue)
#     tuppleFrame['value'] = returnedValues
#     tuppleFrame['predicted'] = returnedPredictions
    return returnedPredictions 

In [38]:
def computeTruth(testingInputs,keepTQDM = True):
    #compute truths for dev set
    trainInfo = pd.read_csv('data/train_info.csv')
    testingInputs['sameArtist'] = pd.Series([isSameArtist(x[0],x[1],trainInfo) for x in tqdm(testingInputs.values,leave=keepTQDM)])
    testingInputs.to_csv('testingResults' + str(fractionOfData) + '.csv')

In [39]:
def getResultsForTestSet(testSet,testFileNames,PercentageTrue,keepTQDM):
    testSet.head()
    testingInputs = pd.DataFrame(list(combinations(testFileNames,2)),columns=['image1','image2'])
    testingInputs.to_csv('testingSet.csv')

    modelPredictions = runProbModel(testingInputs,PercentageTrue)
    testingInputs['predictions'] = modelPredictions
    computeTruth(testingInputs)
    
    return score(testingInputs)

In [43]:
def getAvgScoreForNSplits(n,fraction,fileNamesDF,trainFiles,keepTQDM = True):
    # Look at the non-selected files and do score over random samples
    scores = []
    for i in tqdm(range(n),leave=keepTQDM):
        testingSet=fileNamesDF.drop(trainFiles.index).sample(frac=fraction)
        testFileNames = testingSet.values.flatten()
        scores.append(getResultsForTestSet(testingSet,testFileNames,PercentageTrue,keepTQDM))
#     print(scores)
#     print("Average Score: " + str(np.mean(scores)))
    return scores, np.mean(scores)

In [44]:
getAvgScoreForNSplits(10,0.01,fileNamesDF,trainFiles,False)

HBox(children=(IntProgress(value=0, max=10), HTML(value='')))

TypeError: getResultsForTestSet() takes 3 positional arguments but 4 were given

## Do training and testing over different splits of train and test

In [23]:
def scoreOverRandomTrainingOverRandomSamples(fileNamesDF,nTraining,nScores,fractionTrain,fractionTest,keepTQDM = True):
    scores = []
    trialScores = []
    for i in tqdm(range(nTraining)):
        trainFiles = fileNamesDF.sample(frac=fractionTrain)
        trainFileNames = trainFiles.values.flatten()
        trainingInputs = generateTrainingSet(trainFileNames,fractionTrain,minNumPaintingsPerArtist)
        PercentageTrue = trainingInputs[trainingInputs['sameArtist'] == True].shape[0] / trainingInputs.shape[0]
        PercentageFalse = 1 - PercentageTrue
        
        perTrialScore, overallScore = getAvgScoreForNSplits(nScores,fractionTest,fileNamesDF,trainFiles,keepTQDM = keepTQDM)
        scores.append(overallScore)
        trialScores.append(perTrialScore)
#     getAvgScoreForNSplits(nScores,fraction,fileNamesDF,trainFiles)
    return np.mean(scores), scores, trialScores

In [24]:
meanScoreModel = scoreOverRandomTrainingOverRandomSamples(fileNamesDF,5,10,0.03,0.03,False)[0]

HBox(children=(IntProgress(value=0, max=5), HTML(value='')))

Generating Training Set for 36 files.


HBox(children=(IntProgress(value=0, max=630), HTML(value='')))

HBox(children=(IntProgress(value=0, max=10), HTML(value='')))

HBox(children=(IntProgress(value=0, max=595), HTML(value='')))

...

HBox(children=(IntProgress(value=0, max=595), HTML(value='')))

HBox(children=(IntProgress(value=0, max=595), HTML(value='')))

HBox(children=(IntProgress(value=0, max=595), HTML(value='')))

HBox(children=(IntProgress(value=0, max=595), HTML(value='')))

KeyboardInterrupt: 

In [ ]:
print(meanScoreModel)

## Do training and testing just like before but over different 

## Loading an Image

In [ ]:
filePair = '10.jpg'
jpgfile = Image.open("data/train_1/" + fileName)

In [ ]:
print(jpgfile.getbands())

In [ ]:
imageDF = pd.DataFrame(list(jpgfile.getdata()),columns=['red','green','blue'])

In [ ]:
imageDF['blue'].hist()

## Generating RGB Features

In [ ]:
imgFeatures = pd.concat([imageDF['red'],imageDF['green'],imageDF['blue']])

In [ ]:
dataInfo = pd.read_csv('data/all_data_info.csv')
pixelsX = dataInfo[dataInfo['new_filename'] == fileName]['pixelsx'] # pixelsx and pixelsy could also be features
pixelsY = dataInfo[dataInfo['new_filename'] == fileName]['pixelsy'] # pixelsx and pixelsy could also be features

In [ ]:
imgFeatures = imgFeatures.append(pixelsX)
imgFeatures = imgFeatures.append(pixelsY)

## Creating and training the model

In [ ]:
model = LogisticRegression()
model

# Test

In [ ]:
df = pd.read_csv('data/solution_painter.csv')

In [ ]:
df.describe()